In [2]:
import torchaudio as ta
import pytorch_lightning as pl
import torchaudio.transforms as T
import numpy as np
import random
import torch
import sys
import torch.nn.functional as F
from utils import *
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from IPython.display import Audio, display
from dataset import HighTargetDataset
from tqdm import tqdm



In [18]:
""" Inference LSD, LSDH, STOI """
val_path_gt = "/ssd2/woongzip/Datasets/VCTK_0.92_crop/16k/test"
val_path_target = "/ssd2/woongzip/Datasets/VCTK_0.92_crop/16k_recon/test"
val_path_lr = "/ssd2/woongzip/Datasets/VCTK_0.92_crop/MP3_sr16_8kbps/test"

################ Load Dataset
valid_dataset = HighTargetDataset(val_path_gt, val_path_target, val_path_lr, mode='val')

valid_dataloader = DataLoader(valid_dataset, batch_size = 1, shuffle=False,
                                        num_workers=4, prefetch_factor=2, persistent_workers=True)
lsd_scores = []
lsd_high_scores = []
stoi_scores = []
device = 'cuda'

bar = tqdm(valid_dataloader)
with torch.no_grad():
    for gt, target, lr, name in bar:
        lr = lr.to(device)
        gt = gt.to(device)
        
        lr_np = lr.cpu().numpy()
        gt_np = gt.cpu().numpy()

        # break

        # STOI 계산
        stoi_score = py_stoi(gt_np.squeeze(), lr_np.squeeze(), sample_rate=16000)
        stoi_scores.append(stoi_score)

        # LSD 계산
        lsd_score = lsd_batch(gt_np, lr_np, fs=16000, frame_size=0.02, frame_shift=0.02, start=0, cutoff_freq=8000)
        lsd_scores.append(lsd_score)
        
        # LSD High 계산
        lsd_high_score = lsd_batch(gt_np, lr_np, fs=16000, frame_size=0.02, frame_shift=0.02, start=3000, cutoff_freq=8000)
        lsd_high_scores.append(lsd_high_score)

# 평균 점수 계산 및 출력
avg_lsd = np.mean(lsd_scores)
avg_lsd_high = np.mean(lsd_high_scores)
avg_stoi = np.mean(stoi_scores)

print(f'Average LSD: {avg_lsd:.3f}, Average LSD(high): {avg_lsd_high:.3f}, Average STOI: {avg_stoi:.3f}')


val: GT 3358 / Target 3358 / LR 3358 file numbers loaded!


 63%|██████▎   | 2111/3358 [01:14<00:38, 32.78it/s]